In [ ]:
import geopandas as gpd
import pandas as pd
import os

print("--- Iniciando o script de preparação de dados (PADRÃO ajustado) ---")

# --- 1. CONFIGURAÇÃO INICIAL ---
# ATENÇÃO: Verifique se este caminho relativo está correto para o seu ambiente
caminho_shapefile = 'raw_dataset/BR_Municipios_2024/BR_Municipios_2024.shp'
pasta_saida = 'treated_dataset'
os.makedirs(pasta_saida, exist_ok=True)
print(f"Pasta de saída '{pasta_saida}' garantida.")

try:
    # Lendo o shapefile original, que possui colunas em maiúsculas
    gdf_original = gpd.read_file(caminho_shapefile)
    print(f"Shapefile '{caminho_shapefile}' lido com sucesso. Total de {len(gdf_original)} municípios.")
except Exception as e:
    print(f"ERRO: Não foi possível ler o shapefile. Verifique o caminho: {e}")
    exit()

# --- 2. PREPARAÇÃO DO CENÁRIO 1: TABELA DESNORMALIZADA ---
print("\n--- Processando Cenário 1: Tabela Desnormalizada ---")
# Seleciona apenas as colunas de interesse do GeoDataFrame original
colunas_nao_normalizado = [
    'CD_REGIA', 'NM_REGIA', 'SIGLA_RG', 'CD_UF', 'NM_UF', 'SIGLA_UF',
    'CD_MUN', 'NM_MUN', 'AREA_KM2', 'geometry'
]
df_nao_normalizado = gdf_original[colunas_nao_normalizado].copy()

# Adiciona a coluna de geometria em formato WKT (Well-Known Text)
df_nao_normalizado['geometry_wkt'] = df_nao_normalizado['geometry'].to_wkt()
df_nao_normalizado = df_nao_normalizado.drop(columns=['geometry'])

# Mapeamento explícito do nome original para o nome final desejado
mapa_nomes_nao_normalizado = {
    'CD_REGIA': 'cd_regiao',
    'NM_REGIA': 'nm_regiao',
    'SIGLA_RG': 'sigla_rg',
    'CD_UF': 'cd_uf',
    'NM_UF': 'nm_uf',
    'SIGLA_UF': 'sigla_uf',
    'CD_MUN': 'cd_mun',
    'NM_MUN': 'nm_mun',
    'AREA_KM2': 'area_km2',
    'geometry_wkt': 'geometry_wkt'
}
df_nao_normalizado.rename(columns=mapa_nomes_nao_normalizado, inplace=True)

# Salva o arquivo CSV
caminho_saida_nao_normalizado = os.path.join(pasta_saida, 'cenario_nao_normalizado.csv')
df_nao_normalizado.to_csv(caminho_saida_nao_normalizado, index=False)
print(f"SUCESSO: Arquivo salvo em '{caminho_saida_nao_normalizado}'")

# --- 3. PREPARAÇÃO DO CENÁRIO 2: MODELO DIMENSIONAL ---
print("\n--- Processando Cenário 2: Modelo Dimensional (Fato/Dimensões) ---")

# Tabela de Dimensão: Regiões
print("Gerando Tabela de Dimensão: dim_regioes...")
df_dim_regioes = gdf_original[['CD_REGIA', 'NM_REGIA', 'SIGLA_RG']].drop_duplicates().reset_index(drop=True)
mapa_nomes_regioes = {
    'CD_REGIA': 'id_regiao',
    'NM_REGIA': 'nm_regiao',
    'SIGLA_RG': 'sigla_rg'
}
df_dim_regioes.rename(columns=mapa_nomes_regioes, inplace=True)
caminho_saida_dim_regioes = os.path.join(pasta_saida, 'dim_regioes.csv')
df_dim_regioes.to_csv(caminho_saida_dim_regioes, index=False)
print(f"SUCESSO: Arquivo salvo em '{caminho_saida_dim_regioes}'")

# Tabela de Dimensão: Estados
print("Gerando Tabela de Dimensão: dim_estados...")
df_dim_estados = gdf_original[['CD_UF', 'NM_UF', 'SIGLA_UF', 'CD_REGIA']].drop_duplicates().reset_index(drop=True)
mapa_nomes_estados = {
    'CD_UF': 'id_uf',
    'NM_UF': 'nm_uf',
    'SIGLA_UF': 'sigla_uf',
    'CD_REGIA': 'id_regiao_fk'
}
df_dim_estados.rename(columns=mapa_nomes_estados, inplace=True)
caminho_saida_dim_estados = os.path.join(pasta_saida, 'dim_estados.csv')
df_dim_estados.to_csv(caminho_saida_dim_estados, index=False)
print(f"SUCESSO: Arquivo salvo em '{caminho_saida_dim_estados}'")

# Tabela Fato: Municípios
print("Gerando Tabela Fato: fato_municipios...")
df_fato_municipios = gdf_original[['CD_MUN', 'NM_MUN', 'AREA_KM2', 'CD_UF', 'geometry']].copy()
df_fato_municipios['geometry_wkt'] = df_fato_municipios['geometry'].to_wkt()
df_fato_municipios = df_fato_municipios.drop(columns=['geometry'])
mapa_nomes_municipios = {
    'CD_MUN': 'id_municipio',
    'NM_MUN': 'nm_mun',
    'AREA_KM2': 'area_km2',
    'CD_UF': 'id_uf_fk',
    'geometry_wkt': 'geometry_wkt'
}
df_fato_municipios.rename(columns=mapa_nomes_municipios, inplace=True)
caminho_saida_fato_municipios = os.path.join(pasta_saida, 'fato_municipios.csv')
df_fato_municipios.to_csv(caminho_saida_fato_municipios, index=False)
print(f"SUCESSO: Arquivo salvo em '{caminho_saida_fato_municipios}'")

print("\n--- Processo de preparação de dados concluído! ---")

--- Iniciando o script de preparação de dados (PADRÃO ajustado) ---
Pasta de saída 'treated_dataset' garantida.
Shapefile '../raw_dataset/BR_Municipios_2024/BR_Municipios_2024.shp' lido com sucesso. Total de 5573 municípios.

--- Processando Cenário 1: Tabela Desnormalizada ---
SUCESSO: Arquivo salvo em 'treated_dataset\cenario_nao_normalizado.csv'

--- Processando Cenário 2: Modelo Dimensional (Fato/Dimensões) ---
Gerando Tabela de Dimensão: dim_regioes...
SUCESSO: Arquivo salvo em 'treated_dataset\dim_regioes.csv'
Gerando Tabela de Dimensão: dim_estados...
SUCESSO: Arquivo salvo em 'treated_dataset\dim_estados.csv'
Gerando Tabela Fato: fato_municipios...
SUCESSO: Arquivo salvo em 'treated_dataset\fato_municipios.csv'

--- Processo de preparação de dados concluído! ---
